In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import polyphase
import matplotlib.pyplot as plt
%matplotlib inline

from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from matplotlib.cm import ScalarMappable
from matplotlib import colors

In [ ]:
M = np.ones(4) 
chi = 3.10*np.ones(int(0.5*4*(4-1)))
# M = [5,5,1,1] 
# chi = [1,0.5,0.5,0.5,0.5,1]

f = lambda x : polyphase.flory_huggins(x, M, chi)
engine = polyphase.PHASE(f,50,4)
engine.compute()

In [ ]:
qtplot = polyphase.QuaternaryPlot(engine)
t = 1.0
[fig, axs, cbar] = qtplot.plot_simplices(sliceat=t)
fig.suptitle('Sliced at z={:.2f}'.format(t))
plt.show()


In [ ]:
qtplot = polyphase.QuaternaryPlot(engine)
t = 1.0
[fig, axs, cbar] = qtplot.plot_points(sliceat=t)
fig.suptitle('Sliced at z={:.2f}'.format(t))
plt.show()

In [ ]:
"""3D Contour plot notes

from https://stackoverflow.com/a/32480062

"""


## From solvent solubulities

In [ ]:
import pandas as pd

solvents = pd.read_pickle('../data/solvents.pkl')
polymers = pd.read_pickle('../data/polymers.pkl')
SMs = pd.read_pickle('../data/SM.pkl')

In [ ]:
polymer = polymers[polymers['name']=='PM6']
sm = SMs[SMs['name']=='Y6']
sol1 = solvents[solvents['Solvents']=='Butyl Benzoate']
sol2 = solvents[solvents['Solvents']=='Acetone']

In [ ]:
def get_system(polymer, sm, sol1, sol2):
    delta_sol1 = sol1[['dD','dP','dH']].values[0]
    M_sol1 = 1
    M_ref = sol1['MVol'].values[0]
    
    delta_sol2 = sol2[['dD','dP','dH']].values[0]
    M_sol2 = sol2['MVol'].values[0]/M_ref
    
    delta_sm = sm[['dD','dP','dH']].values[0]
    M_sm = (sm['MW'].values[0]/sm['rho'].values[0])*(1/M_ref)
    
    delta_polymer = polymer[['dD','dP','dH']].values[0]
    M_polymer = (polymer['MW'].values[0]/polymer['rho'].values[0])*(1/M_ref)
    
    M = [M_polymer, M_sm, M_sol1, M_sol2]
    chi,inds = polyphase.get_chi_vector([delta_polymer,delta_sm,delta_sol1, delta_sol2], M_ref, approach=2)
    
    return M, chi, inds

M, chi, inds = get_system(polymer, sm, sol1, sol2)

In [ ]:
f = lambda x : polyphase.flory_huggins(x, M, chi)
engine = polyphase.PHASE(f,50,4)
engine.compute()
engine.plotter()

In [ ]:
from IPython import display
import time

qtplot = polyphase.QuaternaryPlot(engine)
for t in np.linspace(0.05,1,num=10):
    qtplot.plot_simplices(sliceat=t)
    display.display(plt.gcf())
    display.clear_output(wait=True)

In [ ]:
# plot the vertices of simplices
fig, axs = plt.subplots(2,2,subplot_kw={'projection': '3d'}, figsize=(8,8))
axs = axs.flatten()
for ax in axs:
    qtplot.add_outline(ax)
for i, simplex in zip(qtplot.engine.num_comps,qtplot.engine.simplices):
    simplex_vertices = [qtplot.engine.grid[:,x] for x in simplex]
    v = np.asarray([qtplot.from4d23d(vertex) for vertex in simplex_vertices])
    axs[int(i-1)].scatter(v[:,0],v[:,1],v[:,2],color=qtplot.phase_colors[int(i-1)])
    
plt.show()